In [1]:
import time
import datetime

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

import pandas as pd

In [3]:
driver = webdriver.Firefox()

In [4]:
driver.get('http://sharesansar.com/price-history')

companyDropdown = driver.find_element_by_id('company')

try:
    elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'company')))
    print("Located")
except TimeoutException:
    print("Too much time")

Located


In [5]:
select = Select(companyDropdown)
options = [x.text for x in select.options]

In [ ]:
for option in options:
    print('Currently trying to scrape {}'.format(option))
    driver.find_element_by_id('select2-company-container').click()
    driver.find_element_by_tag_name('body').send_keys(option)
    driver.find_element_by_tag_name('body').send_keys(Keys.ENTER)
    
    driver.find_element_by_id('fromdate').clear()
    driver.find_element_by_id('fromdate').send_keys('2011-01-01')
    
    driver.find_element_by_id('todate').clear()
    driver.find_element_by_id('todate').send_keys('2018-05-07')
    
    Select(driver.find_element_by_id('limit')).select_by_value('500')
    
    driver.find_elements_by_tag_name('button')[2].click()
    
    time.sleep(4)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    table = soup.find_all('table')[0]
    
    if (len(table) > 0):  
        tempDf = pd.read_html(str(table),header=0)[0]
        
        if (tempDf.shape[0] > 1):
            df = pd.DataFrame(columns=tempDf.columns)
            df = df.append(tempDf).reset_index(drop=True)

            while (tempDf.shape[0] == 500):
                newStart = df.iloc[-1]['Date']
                newStartdt = datetime.datetime.strptime(newStart, '%M/%d/%Y')
                startConv = newStartdt.strftime('%Y-%m-%d')
                
                driver.find_element_by_id('fromdate').clear()
                driver.find_element_by_id('fromdate').send_keys(startConv)
                
                driver.find_elements_by_tag_name('button')[2].click()

                time.sleep(4)

                soup = BeautifulSoup(driver.page_source, 'lxml')
                table = soup.find_all('table')[0]

                tempDf = pd.read_html(str(table),header=0)[0]
                df = df.append(tempDf).reset_index(drop=True)
            
            tOption = option.replace('/', '-')
            df.to_csv('data/{}.csv'.format(tOption), index=False)
            print('Saved to data/{}.csv'.format(tOption))
            print()

Currently trying to scrape Aakash Capital Limited/ACL
Currently trying to scrape Aankhu Khola Hydropower Company Limited/AANKHU
Currently trying to scrape Aarambha Micro Finance Bittiya Sanstha Limited/AMFI
Saved to data/Aarambha Micro Finance Bittiya Sanstha Limited-AMFI.csv

Currently trying to scrape Ace Capital Limited/ACECAP
Currently trying to scrape Ace Development Bank Limited/ACEDBL
Saved to data/Ace Development Bank Limited-ACEDBL.csv

Currently trying to scrape ACE Development Bank Limited Promoter Share/ACEDPO
Saved to data/ACE Development Bank Limited Promoter Share-ACEDPO.csv

Currently trying to scrape Agricultural Development Bank Limited/ADBL
Saved to data/Agricultural Development Bank Limited-ADBL.csv

Currently trying to scrape Alliance Insurance Company Limited/AIC
Saved to data/Alliance Insurance Company Limited-AIC.csv

Currently trying to scrape Alliance Insurance Company Limited Promoter/AICPO
Saved to data/Alliance Insurance Company Limited Promoter-AICPO.csv



Saved to data/Business Universal Development Bank Limited Promoter Share-BUDBLP.csv

Currently trying to scrape Butwal Finance Limited/BFL
Saved to data/Butwal Finance Limited-BFL.csv

Currently trying to scrape Butwal Power Company Limited/BPCL
Saved to data/Butwal Power Company Limited-BPCL.csv

Currently trying to scrape Butwal Spinning Mills Limited/BSM
Currently trying to scrape Capital Merchant Banking & Finance Limited/CMB
Saved to data/Capital Merchant Banking & Finance Limited-CMB.csv

Currently trying to scrape Capital Merchant Banking & Finance Limited Promotor Share/CMBFLP
Currently trying to scrape Care Ratings Nepal Limited/CARE
Currently trying to scrape CBIL Capital Limited/CBIL
Currently trying to scrape CDS & Clearing Limited/CDS
Currently trying to scrape CEDB Hydro Fund Limited/CEDB
Currently trying to scrape Central Finance Limited/CFCL
Saved to data/Central Finance Limited-CFCL.csv

Currently trying to scrape Central Finance Limited Promoter Share/CFCLPO
Currently

Saved to data/First Microfinance Laghubitta Bittiya Sanstha Limited-FMDBL.csv

Currently trying to scrape Fluer Himalayan Limited/FHL
Currently trying to scrape Foreign Employment Saving Bond - 2077 Ka/NRBFESB77
Currently trying to scrape Forward Community Microfinance Bittiya Sanstha Limited/FOWAD
Saved to data/Forward Community Microfinance Bittiya Sanstha Limited-FOWAD.csv

Currently trying to scrape Ganapati Microfinance Bittiya Sanstha Limited/GMBSL
Currently trying to scrape Gandaki Bikas Bank Limited/GDBL
Saved to data/Gandaki Bikas Bank Limited-GDBL.csv

Currently trying to scrape Gandaki Bikas Bank Limited Promoter Share/GDBLPO
Saved to data/Gandaki Bikas Bank Limited Promoter Share-GDBLPO.csv

Currently trying to scrape Garima Bikas Bank Limited/GBBL
Saved to data/Garima Bikas Bank Limited-GBBL.csv

Currently trying to scrape Garima Bikash Bank Limited Promoter Share /GBBLPO
Saved to data/Garima Bikash Bank Limited Promoter Share -GBBLPO.csv

Currently trying to scrape Garjan

Saved to data/Innovative Development Bank Limited-INDB.csv

Currently trying to scrape International Development Bank Limited/INDBL
Saved to data/International Development Bank Limited-INDBL.csv

Currently trying to scrape International Leasing & Finance Company Limited/ILFC
Saved to data/International Leasing & Finance Company Limited-ILFC.csv

Currently trying to scrape Investa Finance Company Limited/INVESTA
Currently trying to scrape Jalabidyut Lagani Tatha Bikas Company Limited/HIDCL
Saved to data/Jalabidyut Lagani Tatha Bikas Company Limited-HIDCL.csv

Currently trying to scrape Janaki Finance Limited/JFL
Saved to data/Janaki Finance Limited-JFL.csv

Currently trying to scrape Janaki Finance Limited Promoter Share/JFLPO
Saved to data/Janaki Finance Limited Promoter Share-JFLPO.csv

Currently trying to scrape Janasewi Laghubitta Bittiya Sanstha Limited/JANASEWI
Currently trying to scrape Janata Bank Nepal Limited/JBNL
Saved to data/Janata Bank Nepal Limited-JBNL.csv

Currently try

Saved to data/Lumbini Finance Limited Promoter Share-LFLCPO.csv

Currently trying to scrape Lumbini General Insurance Limited/LGIL
Saved to data/Lumbini General Insurance Limited-LGIL.csv

Currently trying to scrape Machhapuchchhre Bank Limited/MBL
Saved to data/Machhapuchchhre Bank Limited-MBL.csv

Currently trying to scrape Machhapuchchhre Bank Limited Promotor Share/MBLPO
Saved to data/Machhapuchchhre Bank Limited Promotor Share-MBLPO.csv

Currently trying to scrape Madhya Bhotekoshi Hydropower Company Limited/MBKHCL
Currently trying to scrape Madhyamanchal Gramin Bikas Bank Limited/MGBL
Currently trying to scrape Madya Paschimanchal Grameen Bikas Bank Limited/MADYAPAS
Currently trying to scrape Maha Laxmi Finance Company Limited Promoter Share/MFLPO
Currently trying to scrape Mahakali Bikas Bank Limited/MBBL
Saved to data/Mahakali Bikas Bank Limited-MBBL.csv

Currently trying to scrape Mahakali Bikas Bank Limited Promoter Share/MBBLPO
Saved to data/Mahakali Bikas Bank Limited Promo

Saved to data/NDEP Development Bank Limited-NDEP.csv

Currently trying to scrape NDEP Development Bank Limited Promoter Share/NDEPPO
Currently trying to scrape Neco Insurace Company Limited Promoter/NILPO
Saved to data/Neco Insurace Company Limited Promoter-NILPO.csv

Currently trying to scrape Neco Insurance Company Limited/NIL
Saved to data/Neco Insurance Company Limited-NIL.csv

Currently trying to scrape Nepal Aawas Finance Limited/NABB
Saved to data/Nepal Aawas Finance Limited-NABB.csv

Currently trying to scrape Nepal Aawas Finance Limited Promoter Share/NABBPO
Saved to data/Nepal Aawas Finance Limited Promoter Share-NABBPO.csv

Currently trying to scrape Nepal Agro Microfinance Bittiya Sanstha Limited/AGRO
Currently trying to scrape Nepal Bangladesh Bank Limited/NBB
Saved to data/Nepal Bangladesh Bank Limited-NBB.csv

Currently trying to scrape Nepal Bangladesh Bank Promoter Share/NBBPO
Saved to data/Nepal Bangladesh Bank Promoter Share-NBBPO.csv

Currently trying to scrape Nepa

Saved to data/NMB Hybrid Fund L- 1-NMBHF1.csv

Currently trying to scrape NMB Microfinance Bittiya Sanstha Limited/NMBMF
Saved to data/NMB Microfinance Bittiya Sanstha Limited-NMBMF.csv

Currently trying to scrape NMB Sulav Investment Fund-1/NMBSF1
Saved to data/NMB Sulav Investment Fund-1-NMBSF1.csv

Currently trying to scrape Nyadi Hydropower Company Limited/NYADI
Currently trying to scrape Office of Company Registrar/REGISTRAR
Currently trying to scrape Old Reliance Finance Limited/Reliance
Currently trying to scrape Om Development Bank Limited/ODBL
Saved to data/Om Development Bank Limited-ODBL.csv

Currently trying to scrape Om Development Bank Limited Promoter Share/ODBLPO
Saved to data/Om Development Bank Limited Promoter Share-ODBLPO.csv

Currently trying to scrape Om Finance Limited/OFL
Saved to data/Om Finance Limited-OFL.csv

Currently trying to scrape Om Finance Limited Promoter Share /OFLPO
Saved to data/Om Finance Limited Promoter Share -OFLPO.csv

Currently trying to scr

Saved to data/Reliable Development Bank Limited Promoter Share-REDBLP.csv

Currently trying to scrape Reliable Finance Company Limited Promoter Share/RFLPO
Saved to data/Reliable Finance Company Limited Promoter Share-RFLPO.csv

Currently trying to scrape Reliable Finance Limited/RIBSL
Saved to data/Reliable Finance Limited-RIBSL.csv

Currently trying to scrape Reliable Nepal Life Insurance Limited/RENLICL
Currently trying to scrape Reliance Finance Limited/RLFL
Saved to data/Reliance Finance Limited-RLFL.csv

Currently trying to scrape Reliance Life Insurance Limited/RLIL
Currently trying to scrape Reliance Lotus Finance Limited Promoter Share/RLFLPO
Saved to data/Reliance Lotus Finance Limited Promoter Share-RLFLPO.csv

Currently trying to scrape Remit Hydropower Limited/REMIT
Currently trying to scrape Resunga Bikas Bank Limited/RBSL
Saved to data/Resunga Bikas Bank Limited-RBSL.csv

Currently trying to scrape Ridi Hydropower Development Company Limited/RHPC
Saved to data/Ridi Hydro

Saved to data/Siddhartha Development Bank Limited Promoter Share-SDBLPO.csv

Currently trying to scrape Siddhartha Equity Fund/SEF
Saved to data/Siddhartha Equity Fund-SEF.csv

Currently trying to scrape Siddhartha Equity Oriented Scheme/SEOS
Saved to data/Siddhartha Equity Oriented Scheme-SEOS.csv

Currently trying to scrape Siddhartha Finance Limited/SFL
Saved to data/Siddhartha Finance Limited-SFL.csv

Currently trying to scrape Siddhartha Finance Limited Promoter share/SFLPO
Saved to data/Siddhartha Finance Limited Promoter share-SFLPO.csv

Currently trying to scrape Siddhartha Insurance Limited/SIL
Saved to data/Siddhartha Insurance Limited-SIL.csv

Currently trying to scrape Siddhartha Insurance Limited Promoter Share/SILPO
Saved to data/Siddhartha Insurance Limited Promoter Share-SILPO.csv

Currently trying to scrape Siddhartha Investment Growth Scheme-1/SIGS1
Saved to data/Siddhartha Investment Growth Scheme-1-SIGS1.csv

Currently trying to scrape Sindhu Bikas Bank Limited/SIND

Saved to data/Unique Finance Limited-UFIL.csv

Currently trying to scrape Unique Finance Limited Promoter Share/UFILPO
Currently trying to scrape Unique Hydel Company Limited/UHCL
Currently trying to scrape United Bikas Bank Limited/UNITEDB
Currently trying to scrape United Finance Limited/UFL
Saved to data/United Finance Limited-UFL.csv

Currently trying to scrape United Finance Limited Promoter Share/UFLPO
Saved to data/United Finance Limited Promoter Share-UFLPO.csv

Currently trying to scrape United Insurance Co. (Nepal) Ltd. Promoter Share/UICPO
Saved to data/United Insurance Co. (Nepal) Ltd. Promoter Share-UICPO.csv

Currently trying to scrape United Insurance Company (Nepal) Limited/UIC
Saved to data/United Insurance Company (Nepal) Limited-UIC.csv

Currently trying to scrape United Modi Hydropower Limited/UMHL
Saved to data/United Modi Hydropower Limited-UMHL.csv

Currently trying to scrape Universal Finance Limited/UFLK
Saved to data/Universal Finance Limited-UFLK.csv

Current